In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_excel('/content/VQA_RAD Dataset Public.xlsx')
df.head(2)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,QID_unique,QID_para,QID_linked,IMAGEID_case,IMAGEID,IMAGEORGAN,EVALUATION,QUESTION,Q_REPHASE,Q_RELATION,Q_FRAMED,Q_TYPE,ANSWER,A_TYPE
0,0,freeform,03f451ca-de62-4617-9679-e836026a7642,https://medpix.nlm.nih.gov/case?id=48e1dd0e-85...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,not evaluated,Are regions of the brain infarcted?,NaN,NaN,NaN,PRES,Yes,CLOSED
1,1,freeform,06e26b2c-04b9-42bc-8e98-1de30a0f7682,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Are the lungs normal appearing?,NaN,NaN,NaN,ABN,No,CLOSED


In [ ]:
rel_col = ['QUESTION', 'Q_FRAMED', 'ANSWER']
s = ""

for col in rel_col:
    for i in range(len(df)):
        if not pd.isnull(df[col][i]):
            s += str(df[col][i]) + " "

# Remove the trailing space at the end of the string
s = s.strip()

In [ ]:
s[0:25]

'Are regions of the brain '

In [ ]:
print(s)

Are regions of the brain infarcted? Are the lungs normal appearing? Is there evidence of a pneumothorax What type of imaging does this not represent? Is this a MRI of the chest? What is not pictured in this image? Is the trachea midline? Is there evidence of an aortic aneurysm? Where is the abnormality? Is there blunting of the costovertebral angles? Is there evidence of an aortic aneurysm? Which organ system is abnormal in this image? Is there airspace consolidation on the left side? Is this an axial plane? Where is the pathology in this image? Is there a pneumothorax? What type of imaging is this? What organ system is pictured? Is there a fracture? How is the patient oriented? Is there swelling of the grey matter? Is there vascular pathology in this image? Is this an anterior posterior image? Is there cardiomyopathy? Is this the brain? Is there air under the diaphragm? What type of image is this? Is this a CT image? Is this an anterior-posterior image In what plane is this image orie

In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_md-0.4.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 9.9 MB/s eta 0:00:00
  Created wheel for en-core-sci-md: filename=en_core_sci_md-0.4.0-py3-none-any.whl size=125733909 sha256=234c023f9fb3c33c9130038fe3257a3105c4596af913b38a8f6c20ea276a5c54
  Stored in directory: /root/.cache/pip/wheels/a6/d7/71/c25351330ce5efecc088e26cffb4dfd90e1a1713a514ffa1db
Successfully built en-core-sci-md
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uni

In [ ]:
import spacy
import requests
import csv

# Load the SciSpaCy model
nlp = spacy.load("en_core_sci_md")

# Define a function to extract medical words from text
def extract_medical_words(text):
    doc = nlp(text)
    medical_words = [token.text for token in doc if token.is_alpha and token.is_stop == False]
    return medical_words

# Example medical text (replace with your own text)
medical_text = s

# Extract medical words
medical_words = extract_medical_words(medical_text)

# Count the frequency of each word
word_frequency = {}
for word in medical_words:
    word_frequency[word] = word_frequency.get(word, 0) + 1

# Sort the words by frequency in descending order
sorted_words = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)

# Get the top 4000 words
top_4000_words = [word[0] for word in sorted_words[:4000]]

# Save to a CSV file
with open("medical_words.csv", "w", newline="", encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Medical Word"])
    csv_writer.writerows([[word] for word in top_4000_words])

print("Top 4000 medical words saved to medical_words.csv")

Top 4000 medical words saved to medical_words.csv


In [ ]:
med_words=pd.read_csv('medical_words.csv')
med_words.head(10)

,Medical Word
0,image
1,located
2,Yes
3,mass
4,patient
5,lesion
6,yes
7,normal
8,right
9,left


Extending the dataset by concatinating the Q_Rephased into the original dataset

In [ ]:
df.head(2)

,QID_unique,QID_para,QID_linked,IMAGEID_case,IMAGEID,IMAGEORGAN,EVALUATION,QUESTION,Q_REPHASE,Q_RELATION,Q_FRAMED,Q_TYPE,ANSWER,A_TYPE
0,0,freeform,03f451ca-de62-4617-9679-e836026a7642,https://medpix.nlm.nih.gov/case?id=48e1dd0e-85...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,not evaluated,Are regions of the brain infarcted?,NaN,NaN,NaN,PRES,Yes,CLOSED
1,1,freeform,06e26b2c-04b9-42bc-8e98-1de30a0f7682,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Are the lungs normal appearing?,NaN,NaN,NaN,ABN,No,CLOSED


In [ ]:
df['Q_REPHASE'].isna().sum()

472

In [ ]:
df2=df.dropna(subset=['Q_REPHASE'])
df2['Q_REPHASE'].isna().sum()

0

In [ ]:
df2['QUESTION']=df2['Q_REPHASE']

<ipython-input-11-a954d59b2aaa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['QUESTION']=df2['Q_REPHASE']


In [ ]:
df3= pd.concat([df,df2])

In [ ]:
len(df3)

4024

In [ ]:
file_path = 'new_extended_dataset.csv'

df3.to_csv(file_path,index=false)

In [ ]:
new_dataset=pd.read_csv('new_extended_dataset.csv')
new_dataset.head(2)

,Unnamed: 0,QID_unique,QID_para,QID_linked,IMAGEID_case,IMAGEID,IMAGEORGAN,EVALUATION,QUESTION,Q_REPHASE,Q_RELATION,Q_FRAMED,Q_TYPE,ANSWER,A_TYPE
0,0,0,freeform,03f451ca-de62-4617-9679-e836026a7642,https://medpix.nlm.nih.gov/case?id=48e1dd0e-85...,https://medpix.nlm.nih.gov/images/full/synpic5...,HEAD,not evaluated,Are regions of the brain infarcted?,NaN,NaN,NaN,PRES,Yes,CLOSED
1,1,1,freeform,06e26b2c-04b9-42bc-8e98-1de30a0f7682,https://medpix.nlm.nih.gov/case?id=b197277b-69...,https://medpix.nlm.nih.gov/images/full/synpic2...,CHEST,not evaluated,Are the lungs normal appearing?,NaN,NaN,NaN,ABN,No,CLOSED


In [ ]:
len(new_dataset)

4024